In [2]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

In [12]:
#收益率分位数分布
def return_distribution(data):       #df:含['日期'、'收盘价']字段的dataframe     rolling_window:窗口大小
    quantile_df = pd.DataFrame()
    rolling_window_l = [1,2,3]
    for i in rolling_window_l:
        df = data.copy()
        df.set_index('日期',inplace=True)
        df = df.sort_index()
        df['滚动收益率(%)'] = np.log(df['收盘价(元)']/df['收盘价(元)'].shift(i))
        df = df[i:]
        df = df[df['滚动收益率(%)']<0]
        df['滚动收益率(%)'] = -df['滚动收益率(%)']  #去掉符号得到下跌绝对值
        df.sort_values(by='滚动收益率(%)',inplace=True)  #按收益率升序排序

         #分位数计算
        quantile_dic = {}  #分位数字典存放最终结果

        for j in [0.5, 0.6, 0.7, 0.8, 0.9, 0.95, 0.99]:
            quantile_dic[str(j)] = df['滚动收益率(%)'].iloc[round(len(df)*j)]   #分位对应排位四舍五入
        quantile_df = pd.concat([quantile_df,pd.Series(quantile_dic).to_frame()],axis=1)
    quantile_df.columns = rolling_window_l
    return quantile_df
    

In [22]:
#成交量分位数分布
def turnover_distribution(df):       
    #数据预处理
    df.sort_values(by='成交量(股)',inplace=True)  #按收益率排序
    #分位数计算
    quantile_dic = {}  #分位数字典存放最终结果
    for i in [0.5, 0.6, 0.7, 0.8, 0.9, 0.95, 0.99]:
        quantile_dic[str(i)] = df['成交量(股)'].iloc[round(len(df)*i)]   #分位对应排位四舍五入
    return quantile_dic   #字典格式，键为概率分位数，值为对应收益率

### 收益率分位数举例

In [21]:
df = pd.read_excel(r'C:\Users\lenovo\Desktop\资料\暑期资料\量化\历史数据\000905收益率.xlsx')
return_distribution(df)

,1,2,3
0.5,0.014185,0.017744,0.025593
0.6,0.017288,0.022455,0.032049
0.7,0.022044,0.027816,0.038521
0.8,0.030374,0.036991,0.052265
0.9,0.047330,0.055529,0.078792
0.95,0.053714,0.087912,0.096392
0.99,0.097368,0.149855,0.144968


### 成交量分位数举例

In [29]:
#600711
df = pd.read_excel(r'C:\Users\lenovo\Desktop\资料\暑期资料\量化\历史数据\000300成交量.xlsx')
quantile_t_dic = turnover_distribution(df)
pd.Series(quantile_t_dic).to_frame()  #键为分位数，值为对应成交量

,0
0.5,14611389600
0.6,15430441100
0.7,16566638200
0.8,17905346000
0.9,20171171900
0.95,23511865800
0.99,28830435200
